This data pertains only to the 2018-2019 hunting season.

In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
# setups up chromedriver and the browser control
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = "http://www.eregulations.com/newjersey/hunting/2018-19-deer-harvest-summaries/"

# sets browser to visit the url and then wait to ensure all of the page loads
browser.visit(url)
time.sleep(5)

# builds soup object for data scraping
html = browser.html
soup = BeautifulSoup(html, "html.parser")

In [9]:
# pulls in table from the soup object
table = soup.find_all("table", class_="New-Tables")
# converts the table into pandas dataframe
hunt_df = pd.read_html(str(table))[0]
# drops row populated by table title
hunt_df = hunt_df.drop([0])
# moves row with table headers into column headers
hunt_df.columns = hunt_df.iloc[0]
hunt_df = hunt_df.drop([1])
# drop totals row
hunt_df = hunt_df.drop([62])
# drop unneeded harvest breakdown by permit type
del hunt_df["Fall Bow"]
del hunt_df["Permit Bow"]
del hunt_df["Six-day Firearm"]
del hunt_df["Permit Muzzleloader"]
del hunt_df["Permit Shotgun"]
del hunt_df["Winter Bow"]
del hunt_df["Youth Days"]
# convert DMZ and harvest into int64
hunt_df["DMZ"] = hunt_df["DMZ"].astype("int64")
hunt_df["Total Harvest"] = hunt_df["Total Harvest"].astype("int64")

In [10]:
# assign each DMZ to a county
# read in ref csv
dmz_county = "../CDC_input/dmz_county.csv"
# convert to pandas dataframe
ref_df = pd.read_csv(dmz_county)
# converts DMZ column into int64 for later merge
ref_df["DMZ"] = ref_df["DMZ"].astype("int64")
# merge county data into primary dataframe
hunt_df = hunt_df.merge(ref_df, on="DMZ")

In [11]:
# rename and reorder columns
hunt_df.columns = ["dmz","total_harvest","county"]
hunt_df = hunt_df[list(("dmz","county","total_harvest"))]
hunt_df.head()

,dmz,county,total_harvest
0,1,Sussex County,443
1,2,Sussex County,1596
2,3,Passaic County,360
3,4,Sussex County,393
4,5,Warren County,2444


In [13]:
# export as csv
hunt_df.to_csv(r"../CDC_output/2018-19_deer_harvest.csv")

In [12]:
hunt_df.dtypes

dmz               int64
county           object
total_harvest     int64
dtype: object